In [1]:
import numpy as np
import scipy.linalg as la

**QUESTION 1**

In [2]:
def a_solve(a):
    M = np.array([[4,5,7],[0,1,9],[1,3,7]])
    b = np.array([a,1-a,1])
    return la.solve(M,b)

**QUESTION 2**

In [3]:
# SOLUTION USING FANCY INDEXING

def minor_det(A,i,j):
    '''Compute the determinant of A_ij where A_ij is the matrix
    obtained from A by removing the ith row and jth column.'''
    nrows, ncols = A.shape
    row_index = np.arange(0,nrows)
    col_index = np.arange(0,ncols)
    A_i = A[row_index != i, :]
    A_ij = A_i[:, col_index !=j]
    return la.det(A_ij)

In [4]:
A = np.array([[4,5,7],[0,1,9],[1,3,7]])
minor_det(A,0,1)

-9.0

In [5]:
B = np.arange(0,9).reshape(3,3)
minor_det(B,2,2)

-3.0

In [6]:
# SOLUTION USING .delete METHOD

def minor_det(A,i,j):
    '''Compute the determinant of A_ij where A_ij is the matrix obtained from A
    by removing the ith row and jth column.'''
    # Create a copy of A so that we don't modify the original matrix A
    B = np.copy(A)
    B_i = np.delete(B,i,0)
    B_ij = np.delete(B_i,j,1)
    return la.det(B_ij)

In [7]:
A = np.array([[4,5,7],[0,1,9],[1,3,7]])
minor_det(A,0,1)

-9.0

In [8]:
B = np.arange(0,9).reshape(3,3)
minor_det(B,2,2)

-3.0

**QUESTION 3**

Notice that the outer product $u^T v$ has only one nonzero eigenvalue given by the inner product $u v^T$.

In [9]:
# SOLUTION USING MATRIX MULTIPLICATION

def outer_eig(u,v):
    '''Compute the eigenvalues of the outer product u.T @ v'''
    u = np.array(u)
    v = np.array(v)
    A = u.reshape(len(u),1) @ v.reshape(1,len(v))
    evals, evecs = la.eig(A)
    return evals

In [10]:
u = np.array([3,1,0])
v = np.array([-2,0,1])
outer_eig(u,v)

array([ 0.+0.j, -6.+0.j,  0.+0.j])

In [11]:
u @ v

-6

In [12]:
# SOLUTION USING np.outer FUNCTION

def outer_eig(u,v):
    '''Compute the eigenvalues of the outer product u.T @ v'''
    u = np.array(u)
    v = np.array(v)
    A = np.outer(u,v)
    evals, evecs = la.eig(A)
    return evals

In [13]:
outer_eig(u,v)

array([ 0.+0.j, -6.+0.j,  0.+0.j])

**QUESTION 4**

In [14]:
# SOLUTION USING MATRIX MULTIPLICATION

def elementary(A,L,i,j,row=True):
    '''Perform elementary operation adding L times row/column i to row/column j.'''
    A = np.array(A)
    nrows, ncols = A.shape
    if row:
        M = np.eye(nrows)
        if i == j:
            M[i,j] = L + 1
        else:
            M[j,i] = L
        return M @ A
    else:
        M = np.eye(ncols)
        if i == j:
            M[i,j] = L + 1
        else:
            M[i,j] = L
        return A @ M

In [15]:
elementary(np.eye(2),1,1,1)

array([[ 1.,  0.],
       [ 0.,  2.]])

In [16]:
elementary(np.eye(2),1,1,0)

array([[ 1.,  1.],
       [ 0.,  1.]])

In [17]:
elementary(np.eye(2),2,0,1,row=False)

array([[ 1.,  2.],
       [ 0.,  1.]])

In [18]:
elementary(np.arange(0,16).reshape(4,4),2,0,3,row=False)

array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,  15.],
       [  8.,   9.,  10.,  27.],
       [ 12.,  13.,  14.,  39.]])

In [19]:
# SOLUTION USING ROW ASSIGNMENT

def elementary(A,L,i,j,row=True):
    '''Perform elementary operation adding L times row/column i to row/column j.'''
    # Create a copy of A so that we don't modify the original matrix A
    B = np.copy(np.array(A))
    nrows, ncols = B.shape
    if row:
        B[j,:] = L*B[i,:] + B[j,:]
        return B
    else:
        B[:,j] = L*B[:,i] + B[:,j]
        return B

In [20]:
elementary(np.eye(2),2,0,1,row=False)

array([[ 1.,  2.],
       [ 0.,  1.]])

In [21]:
elementary(np.arange(0,16).reshape(4,4),2,0,3,row=False)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6, 15],
       [ 8,  9, 10, 27],
       [12, 13, 14, 39]])

**QUESTION 5**

In [22]:
# SOLUTION USING PROJECTION ONTO THE NORMAL VECTOR

def nearest_point(coeffs,P):
    '''Find point on plane ax + by + cz = d closest to P.'''
    a,b,c,d = coeffs
    # Find a point R on the plane
    if a != 0:
        R = np.array([d/a,0,0])
    elif b != 0:
        R = np.array([0,d/b,0])
    elif c != 0:
        R = np.array([0,0,d/c])
    else:
        print('Not all coefficients can be zero.')
        return None
    P = np.array(P)
    RP = P - R
    n = np.array([a,b,c])
    # The nearest point is P minus the projection of RP onto n
    return P - (RP @ n) / (n @ n) * n

In [23]:
nearest_point([0,0,1,0],[1,0,1])

array([ 1.,  0.,  0.])

In [24]:
# SOLUTION USING THE LINE FROM P IN THE DIRECTION OF THE NORMAL VECTOR

def nearest_point(coeffs,P):
    '''Find point on plane ax + by + cz = d closest to P.'''
    P = np.array(P)
    a,b,c,d = coeffs
    N = np.array([a,b,c])
    if [a,b,c] == [0,0,0]:
        print('Not all coefficients can be zero.')
        return None
    # Line from P to plane in the direction N normal to the plane
    # is given by P + t*N and the value of t which gives the
    # intersection of the line and the plane satisfies N.(P + t*N) = d
    t = (d - (N @ P))/(N @ N)
    return P + t*N

In [25]:
nearest_point([0,0,1,0],[1,0,1])

array([ 1.,  0.,  0.])

In [26]:
nearest_point([0,0,0,0],[1,1,1])

Not all coefficients can be zero.
